### Chain Using LangGraph

In this section we will see how we can build a simple chain using Langgraph that uses 4 important concepts:

* How to use chat messages as our graph state
* How to use chat models in graph nodes
* How to bind tools to our LLM in chat models
* How to execute the tools call in our graph nodes
---

### How to use chat messages as our graph state


#### 1. Messages

We can use messages which can be used to capture different roles within a conversation. LangChain has various messages types including HumanMessage(), AIMessage(), SystemMessage() and ToolMessage(). These represent a message from the user, from chat model, for the chat model to instruct behaviour, and from a tool call.

Every message have these important components:

* content - content of the message
* name - specify the name of the author
* response_metadata - optionally, a dict of metadata(e.g., often populated by model provider for AIMessages)

In [1]:
from langchain_core.messages import HumanMessage, AIMessage
from pprint import pprint

messages = [AIMessage(content = "How can i help you?", name="AI")]
messages.append(HumanMessage(content="I want to learn Machine Learning.", name="Kalyan"))
messages.append(AIMessage(content="Which specific topic you want to learn?", name="AI"))

for message in messages:
    message.pretty_print()

================================== Ai Message ==================================
Name: AI

How can i help you?
================================ Human Message =================================
Name: Kalyan

I want to learn Machine Learning.
================================== Ai Message ==================================
Name: AI

Which specific topic you want to learn?


##### *We will learn how to append all these different types of messages into our graph state in a specific manner using Reducers in LangGraph*
---

### 2. Chat Models

We can use the sequence of messages as input with the chatmodels using LLM's and OPENAI / GROQ / GEMINI models.


In [6]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model = "gemma3:270m")

messages.append(HumanMessage(content="I want to know about Principal Component Analysis and how that is executed or used. no need of code. only explain me clearly on how to use that in 500 words."))
llm.invoke(messages).content

"Okay, let's dive into Principal Component Analysis (PCA). It's a powerful technique used in machine learning to reduce dimensionality and improve the performance of various machine learning models.\n\nPCA is a dimensionality reduction technique that aims to find the most important components of a dataset. It essentially identifies the most significant features that capture the underlying patterns and relationships within the data. By reducing the number of features, PCA can often lead to significant improvements in model accuracy, especially when dealing with high-dimensional data.\n\nThe core idea behind PCA is to take the original data and transform it into a lower-dimensional space. This lower-dimensional space represents the most important features, which are the components that are most relevant to the problem. The transformation applied to these features can then be used to create new, simpler features that capture the essence of the data.\n\nTo implement PCA, you typically need


### Tools

Tools can be integrated with the LLM moddels to interat with external systems. External systems can be API's, third party tools or functions. 

Whenever a query is asked to the model, it can choose to call the tool and this query is based on the natural language input and this will return an output that matches the tool's schema

Now when a tool is used along with the llm, the llm should be able to understand what the tool is and when to use that tool. This can be achieved with the special function / property called `bind_tools` which is used along with llm. 


This `bind_tools` property works according to the doc-string that we describe during creation of tools.

In [81]:
from datetime import datetime
def date():
    """ Returns the current date, month and year
    args: none
    return: today's date,month and year of datetime object
    """
    return datetime.now().date()


In [82]:
tools = [date]

Here, I am using llama3.2 as it is the only downloadable model which supports the tool_binding function in Ollama.

In [63]:
llm = ChatOllama(model = "llama3.2")
llm_tools = llm.bind_tools(tools)

In [64]:
llm.invoke("what is the current date and time?")

AIMessage(content="I'm not currently able to share the date and time.", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-09-28T08:02:38.3355673Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8554221300, 'load_duration': 236250500, 'prompt_eval_count': 33, 'prompt_eval_duration': 6194686000, 'eval_count': 13, 'eval_duration': 2111333700, 'model_name': 'llama3.2'}, id='run--82ae0351-2999-4ab7-8c69-d3692c481f13-0', usage_metadata={'input_tokens': 33, 'output_tokens': 13, 'total_tokens': 46})

In [65]:
llm_tools.invoke("what is the current date and time?")

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-09-28T08:03:01.12756Z', 'done': True, 'done_reason': 'stop', 'total_duration': 22745172800, 'load_duration': 194444100, 'prompt_eval_count': 221, 'prompt_eval_duration': 20351862200, 'eval_count': 12, 'eval_duration': 2195700400, 'model_name': 'llama3.2'}, id='run--8ec0df0b-1179-4b8a-910c-4a3da06e11e2-0', tool_calls=[{'name': 'date', 'args': {}, 'id': 'd63ab087-8501-4a90-8384-40de27679948', 'type': 'tool_call'}], usage_metadata={'input_tokens': 221, 'output_tokens': 12, 'total_tokens': 233})

In [66]:
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage

class State(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

In [67]:
def llm_call(state:State):
    return {"messages":[llm_tools.invoke(state["messages"])]}

In [73]:
from langgraph.graph import START,END,StateGraph

builder = StateGraph(State)

builder.add_node("llm_tool",llm_call)

builder.add_edge(START,"llm_tool")
builder.add_edge("llm_tool",END)

builder_graph = builder.compile()

In [74]:
response = builder_graph.invoke(
    {
        'messages':"what is today's date?"
    }
)

In [75]:
for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

what is today's date?
================================== Ai Message ==================================
Tool Calls:
  date (e6f9ac32-6f7f-4106-8212-a41ed859f950)
 Call ID: e6f9ac32-6f7f-4106-8212-a41ed859f950
  Args:


Here, the ai is not giving any output. It is using the tools, but the output after the tool has been used is not displayed back. This is probably because we didn't defined any node for the tools in our graph above.

 And LangGraph provides a function named `ToolNode` to add the tool node. For condition, we have `tools_condition` in LangGraph which when added to conditional edge of our graph, either directs to the tool node if the llm asks for it, or directs to the END node after the llm response has been generated.

In [83]:
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

builder = StateGraph(State)

builder.add_node("llm_tool",llm_call)
builder.add_node("tools",ToolNode(tools))

builder.add_edge(START,"llm_tool")
builder.add_conditional_edges("llm_tool",tools_condition)
builder.add_edge("tools",END)

builder_graph = builder.compile()

In [84]:
response = builder_graph.invoke({"messages":"today's date?"})

for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

today's date?
================================== Ai Message ==================================
Tool Calls:
  date (61db6710-5338-4082-8e08-241074372b41)
 Call ID: 61db6710-5338-4082-8e08-241074372b41
  Args:
================================= Tool Message =================================
Name: date

2025-09-28
